In [1]:
# %%capture captured_output
# 实验环境已经预装了mindspore==2.3.0，如需更换mindspore版本，可更改下面 MINDSPORE_VERSION 变量
!pip uninstall mindspore -y
!export MINDSPORE_VERSION=2.3.0
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/${MINDSPORE_VERSION}/MindSpore/unified/aarch64/mindspore-${MINDSPORE_VERSION}-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.mirrors.ustc.edu.cn/simple

Found existing installation: mindspore 2.3.0
Uninstalling mindspore-2.3.0:
  Successfully uninstalled mindspore-2.3.0
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.2/273.2 MB 81.5 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install jieba -i https://pypi.mirrors.ustc.edu.cn/simple
!pip install tiktoken -i https://pypi.mirrors.ustc.edu.cn/simple
%env HF_ENDPOINT=https://hf-mirror.com

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 72.8 MB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314458 sha256=aea641b9a1f6fb67e643986fe5190e8d3d5076bf6a48a3ee0ca21844e68e456e
  Stored in directory: /home/mindspore/.cache/pip/wheels/06/74/c0/43b175e9228f8213c84801781a2afd74e1453f94677ca44004
Successfully built jieba
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.1/777.1 kB 90.3 MB/s eta 0:00:00
env: HF_ENDPOINT=https://hf-mirror.com


In [3]:
!wget https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/model/ZhipuAI/glm-4-9b-chat.zip
!mkdir -p ./.mindnlp/model/ZhipuAI
!unzip -d ./.mindnlp/model/ZhipuAI/ glm-4-9b-chat.zip

--2024-08-29 07:13:10--  https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/model/ZhipuAI/glm-4-9b-chat.zip
Resolving mindspore-demo.obs.cn-north-4.myhuaweicloud.com (mindspore-demo.obs.cn-north-4.myhuaweicloud.com)... 100.125.224.5
Connecting to mindspore-demo.obs.cn-north-4.myhuaweicloud.com (mindspore-demo.obs.cn-north-4.myhuaweicloud.com)|100.125.224.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14885997618 (14G) [application/zip]
Saving to: ‘glm-4-9b-chat.zip’

glm-4-9b-chat.zip   100%[===================>]  13.86G  72.6MB/s    in 6m 52s  

2024-08-29 07:20:02 (34.5 MB/s) - ‘glm-4-9b-chat.zip’ saved [14885997618/14885997618]

Archive:  glm-4-9b-chat.zip
   creating: ./.mindnlp/model/ZhipuAI/glm-4-9b-chat/
  inflating: ./.mindnlp/model/ZhipuAI/glm-4-9b-chat/model-00003-of-00010.safetensors  
  inflating: ./.mindnlp/model/ZhipuAI/glm-4-9b-chat/model-00007-of-00010.safetensors  
  inflating: ./.mindnlp/model/ZhipuAI/glm-4-9b-chat/model.safetensors.i

In [5]:
import argparse
import os

import mindspore
from mindspore.common.api import _no_grad

from tqdm import tqdm
from mindnlp.core.optim import AdamW
from mindnlp import evaluate
from mindnlp.dataset import load_dataset
from mindnlp.engine import set_seed
from mindnlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup
from mindnlp.peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    LoraConfig,
    PromptEncoderConfig,
    PeftConfig,
    PeftModel,
)

/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the default dict

In [6]:
import mindspore
import mindnlp
from mindnlp.core import no_grad
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from mindspore._c_expression import _framework_profiler_step_start
from mindspore._c_expression import _framework_profiler_step_end
# 加载tokenizer文件

In [17]:
batch_size = 32
model_name_or_path = "./.mindnlp/model/ZhipuAI/glm-4-9b-chat"
task = "mrpc"
# peft_type = PeftType.PROMPT_TUNING
peft_type = PeftType.P_TUNING
num_epochs = 20
# peft_config = LoraConfig(task_type="SEQ_2_SEQ_LM", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
peft_config = PromptEncoderConfig(task_type="SEQ_2_SEQ_LM", num_virtual_tokens=20, encoder_hidden_size=128)
lr = 3e-4
max_length = 128

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, mirror='modelscope')
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    mirror='modelscope',
    ms_dtype=mindspore.float16,
).eval()

if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
model = get_peft_model(model, peft_config)
# print(model)
model.print_trainable_parameters()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

trainable params: 2,785,280 || all params: 9,402,736,640 || trainable%: 0.02962201438410169


In [11]:
import mindspore.dataset as ds
dataset = ds.CSVDataset(dataset_files="./dataset/output.csv", column_names=['questions', 'answers'], column_defaults=[])
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])
# next(train_dataset.create_dict_iterator())
# def add_text_label(ask, answer, label):
#     return ask, answer, classes[label.item()]

# train_dataset = train_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])
# validation_dataset = validation_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])

# def add_text_label(ask, label):
#     return ask, label, classes[label.item()]

# train_dataset = train_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])
# validation_dataset = validation_dataset.map(add_text_label, ['ask', 'answer', 'department'], ['ask', 'answer', 'text_label'])

[WARNING] ME(153:281472832564096,MainProcess):2024-08-29-07:57:53.221.580 [mindspore/dataset/engine/datasets.py:1217] Dataset is shuffled before split.


In [12]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, inputs, outputs):
        lock.acquire()
        model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(outputs, max_length=max_length, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['questions', 'answers']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [15]:
import mindnlp
optimizer = mindnlp.core.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [ ]:
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = mindspore.value_and_grad(forward_fn, None, tuple(model.parameters()))

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|          | 0/169 [00:00<?, ?it/s]